# Gas reserves  vs Gas production 

Data from [BP statistical review of world energy](https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html).

**Note:** I added to the original BP Excel file, a sheet with the codes and regions numbers of the countries in the BP dataset. This allows coloring the countries by region, and labeling them by code. See Excel file.

In [ ]:
# import libraries
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# primary energy consumption
# file path
path = "../data/bp-stats-review-2022-all-data.xlsx"
# read data from third row and drop last 14 rows
pec = pd.read_excel(path, sheet_name = "Primary Energy Consumption", header=2, skipfooter=14) 
# remove empty rows
pec.dropna(inplace=True)
# remove rows containing "Total"
pec.drop(pec[pec["Exajoules"].str.contains("Total")].index, inplace=True)
# remove last three columns
pec.drop(columns=pec.columns[-3:], axis=1,  inplace=True)
# make first column the index of the DataFrame
pec.set_index("Exajoules", inplace=True)
# Number of rows should be 92
print("Number of rows =", len(pec.index))

pec.tail()

In [ ]:
# primary energy consumption per capita
# read data from third row and drop last 13 rows
pec_cap = pd.read_excel(path, sheet_name = "Primary Energy - Cons capita", header=2, skipfooter=13) 
# remove empty rows
pec_cap.dropna(inplace=True)
# remove rows containing "Total"
pec_cap.drop(pec_cap[pec_cap["Gigajoule per capita"].str.contains("Total")].index, inplace=True)
# remove last two columns
pec_cap.drop(columns=pec_cap.columns[-2:], axis=1,  inplace=True)
# make first column the index of the DataFrame
pec_cap.set_index("Gigajoule per capita", inplace=True)
# Number of rows should be 92
print("Number of rows =", len(pec_cap.index))

pec_cap.tail()

In [ ]:
# gas production
# read data from third row and drop last 16 rows
prod = pd.read_excel(path, sheet_name = "Gas Production - Bcm", header=2, skipfooter=16) 
# remove empty rows
prod.dropna(inplace=True)
# remove rows containing "Total"
prod.drop(prod[prod["Billion cubic metres"].str.contains("Total")].index, inplace=True)
# remove last three columns
prod.drop(columns=prod.columns[-3:], axis=1,  inplace=True)
# make first column the index of the DataFrame
prod.set_index("Billion cubic metres", inplace=True)
# Number of rows should be 56
print("Number of rows =", len(prod.index))

prod.tail()

In [ ]:
# gas reserves
# read data from third row and drop last 19 rows
res = pd.read_excel(path, sheet_name = "Gas - Proved reserves history ", header=2, skipfooter=19) 
# remove empty rows
res.dropna(how="all",inplace=True)
# fill nans with zeros
res.fillna(0, inplace=True)
# remove rows containing "Total"
res.drop(res[res["Trillion cubic metres"].str.contains("Total")].index, inplace=True)
# remove last three columns
res.drop(columns=res.columns[-3:], axis=1,  inplace=True)
# make first column the index of the DataFrame
res.set_index("Trillion cubic metres", inplace=True)
# use only the indexes/countries in the gas production DataFrame
res = res.loc[prod.index]
# set the name of the axis for the index to Trillion cubic meters
res.rename_axis("Trillion cubic metres", inplace=True)
# Number of rows should be 56
print("Number of rows =", len(res.index))

res.tail()

In [ ]:
# codes and regions
cod_reg = pd.read_excel(path, sheet_name = "Codes and regions") 
# make first column the index of the DataFrame
cod_reg.set_index("Country", inplace=True)
# use only the indexes/countries in the gas production DataFrame
cod_reg = cod_reg.loc[prod.index]
# set the name of the axis for the index to ""
cod_reg.rename_axis("", inplace=True)
# Number of rows should be 56
print("Number of rows =", len(cod_reg.index))

cod_reg.tail()

In [ ]:
# compute population by dividing primary energy consumption pec,
# by primary energy_consumption per capita pec_cap
# Notice that pec is in Exajoules, while pec_cap is in Gigajoules
# Therefore population in millions is
population = (pec*1000)/pec_cap

In [ ]:
# Add missing countries to population

# Read World Bank population database
path = "../data/WorldBankPopulation.xls"
# read data from third row and drop last 14 rows
wbp = pd.read_excel(path, sheet_name = "Data", header=3) 
# make first column the index of the DataFrame
wbp.set_index("Country Name", inplace=True)
# remove first eight columns
wbp.drop(columns=wbp.columns[0:8], axis=1,  inplace=True)
# make wbp columns equal to population columns
wbp.columns = population.columns
# rename some countries
wbp.rename(index={"Syrian Arab Republic":"Syria", "Yemen, Rep.":"Yemen", 
                  "Congo, Dem. Rep.":"Republic of Congo", "Brunei Darussalam":"Brunei"}, inplace=True)
# select missing countries 
wbp = wbp.loc[["Bolivia", "Bahrain", "Syria", "Yemen", "Angola", "Chad", "Republic of Congo", "Equatorial Guinea", 
               "Gabon", "Libya", "Nigeria", "South Sudan", "Sudan", "Tunisia", "Brunei", "Myanmar"]]
# convert population to millions
wbp = wbp / 1e6
# add missing countries
population = pd.concat([population, wbp])

In [ ]:
# There can be errors in the computed populations here
# but I believe this is the best way to compute the "Other" areas populations

# Add population of Other areas
population.loc["Other S. & Cent. America"] = population.loc[["Central America", "Other Caribbean", 
                                                             "Other South America"]].sum()
population.loc["Other Africa"] = population.loc[["Eastern Africa", "Middle Africa", "Western Africa", 
                                                 "Other Northern Africa", "Other Southern Africa"]].sum()
# subtract to Other areas
population.loc["Other S. & Cent. America"] = population.loc["Other S. & Cent. America"] \
    - population.loc["Bolivia"] 

population.loc["Other Middle East"] = population.loc["Other Middle East"] - - population.loc["Bahrain"] \
    - population.loc["Syria"] - population.loc["Yemen"]

population.loc["Other Africa"] = population.loc["Other Africa"] - population.loc["Angola"] \
    - population.loc["Chad"] - population.loc["Republic of Congo"] - population.loc["Equatorial Guinea"] \
    - population.loc["Gabon"] - population.loc["Libya"] - population.loc["Nigeria"] \
    - population.loc["South Sudan"] - population.loc["Sudan"] - population.loc["Tunisia"]

population.loc["Other Asia Pacific"] = population.loc["Other Asia Pacific"] - population.loc["Brunei"] \
    - population.loc["Myanmar"]

# use only the indexes/countries in the gas production DataFrame
population = population.loc[prod.index]
# set the name of the axis for the index to Millions
population.rename_axis("Millions", inplace=True)
# Number of rows should be 56
print("Number of rows =", len(population.index))

population.tail()

In [ ]:
# check the indexes of the DataFrames are equal
print(res.index.equals(prod.index))
print(res.index.equals(population.index))
print(res.index.equals(cod_reg.index))

In [ ]:
# graph as scatter the gas reserves versus gas production for the year 2020
# color the points by region and make their size proportional to population

# regions:
# 1 = North America
# 2 = South and Central America
# 3 = Europe
# 4 = CIS
# 5 = Middle East
# 6 = Africa
# 7 = Asia Pacific
regions = [1, 2, 3, 4, 5, 6, 7]
regions = regions[::-1] # reverse list of regions

# colors for regions
colors = ["palegreen", "darkgreen", "blue", "magenta", "orange", "red", "yellow"]
colors = colors[::-1] # reverse list of colors

# year
year = 2020

# make figure
fig, ax = plt.subplots(figsize=(15,7.5))

# for each region
for (region, color) in zip(regions, colors):
    # extract region data
    my_res = res[cod_reg["region"] == region]
    my_prod = prod[cod_reg["region"] == region]
    my_population = population[cod_reg["region"] == region]
    # plot data
    ax.scatter(my_res[year], my_prod[year], s=my_population[year]*2, 
               c=color, edgecolor="0", alpha=0.75, zorder=2)
    # plot labels
    for index in my_res.index:
        if my_res.loc[index,year] >= 0.1 and my_prod.loc[index,year] >= 0.1:
            ax.text(x=my_res.loc[index,year], y=my_prod.loc[index,year], 
                    s=cod_reg.loc[index,"code"], size=8, zorder=3)

# plot year
ax.text(x = 0.3, y = 1.5, s=str(year), 
        fontdict=dict(fontfamily="Courier New", color="lightgray", size=250), zorder=1)    

# set axes
ax.set_xlim([0.1, 50])
ax.set_ylim([0.1, 2_000])
ax.set_xscale("log") # x axis is log
ax.set_yscale("log") # y axis is log
ax.set_xlabel("Gas reserves [Trillion cubic metres]")
ax.set_ylabel("Gas production [Billion cubic metres]")
ax.grid(True)

In [ ]:
# run this cell to install celluloid
import sys
!{sys.executable} -m pip install celluloid

In [ ]:
# Create animation of gas reserves versus gas production  over time

# import celluloid Camera
from celluloid import Camera

# create figure
fig, ax = plt.subplots(figsize=(15,7.5))
# set axes
ax.set_xlim([0.1, 50])
ax.set_ylim([0.1, 2_000])
ax.set_xscale("log") # x axis is log
ax.set_yscale("log") # y axis is log
ax.set_xlabel("Gas reserves [Trillion cubic metres]")
ax.set_ylabel("Gas production [Billion cubic metres]")
ax.grid(True)
# create camera
camera = Camera(fig)

# for each year
for year in res.columns:
    # for each region
    for (region, color) in zip(regions, colors):
        # extract region data
        my_res = res[cod_reg["region"] == region]
        my_prod = prod[cod_reg["region"] == region]
        my_population = population[cod_reg["region"] == region]
        # plot data
        ax.scatter(my_res[year], my_prod[year], s=my_population[year]*2, 
                   c=color, edgecolor="0", alpha=0.75, zorder=2)
        # plot labels
        for index in my_res.index:
            if my_res.loc[index,year] >= 0.1 and my_prod.loc[index,year] >= 0.1:
                ax.text(x=my_res.loc[index,year], y=my_prod.loc[index,year], 
                        s=cod_reg.loc[index,"code"], size=8, zorder=3)
    # plot year
    ax.text(x = 0.3, y = 15, s=str(year), 
            fontdict=dict(fontfamily="Courier New", color="silver", size=250), zorder=1)
    # snap current plot
    camera.snap()

To play the animation in the notebook, you may need to install ffmpeg. For macOS, follow [this link](https://phoenixnap.com/kb/ffmpeg-mac)

In [ ]:
# import HTML to display video in notebook
from IPython.display import HTML
# create animation
animation = camera.animate(interval = 500, repeat = True, repeat_delay = 500)
# play animation
HTML(animation.to_html5_video())

In [ ]:
# save animation
animation.save("../movies/GasResVsProd.mp4", dpi=300)